In [650]:
!pip install -r ../requirements.txt

     |████████████████████████████████| 1.3 MB 2.6 MB/s eta 0:00:01


## Imports

In [781]:
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import pandas as pd
import time
import seaborn as sns
from sklearn import metrics
from sklearn.decomposition import PCA
from collections import Counter
from imblearn.over_sampling import SMOTE

In [782]:
path = '../datasets/diabetes_data_initial.csv'
data = pd.read_csv(path)

## Age

In [783]:
replaceDict = {'[0-10)' : 5,
'[10-20)' : 15,
'[20-30)' : 25,
'[30-40)' : 35,
'[40-50)' : 45,
'[50-60)' : 55,
'[60-70)' : 65,
'[70-80)' : 75,
'[80-90)' : 85,
'[90-100)' : 95}

data['age'] = data['age'].apply(lambda x : replaceDict[x])


## Patient History

In [784]:
data['prev_visits'] = data.groupby('patient_nbr').encounter_id.transform(lambda x: range(len(x)))

## Medical Specialty

In [785]:
high_frequency = ['InternalMedicine', 'Family/GeneralPractice', 'Cardiology', 'Surgery-General', 'Orthopedics', 'Orthopedics-Reconstructive', 'Emergency/Trauma', 'Urology','ObstetricsandGynecology','Psychiatry','Pulmonology ','Nephrology','Radiologist']
low_frequency = ['Surgery-PlasticwithinHeadandNeck','Psychiatry-Addictive','Proctology','Dermatology','SportsMedicine','Speech','Perinatology', 'Neurophysiology','Resident','Pediatrics-Hematology-Oncology','Pediatrics-EmergencyMedicine','Dentistry','DCPTEAM','Psychiatry-Child/Adolescent', 'Pediatrics-Pulmonology','Surgery-Pediatric','AllergyandImmunology','Pediatrics-Neurology','Anesthesiology','Pathology','Cardiology-Pediatric', 'Endocrinology-Metabolism','PhysicianNotFound','Surgery-Colon&Rectal','OutreachServices', 'Surgery-Maxillofacial','Rheumatology','Anesthesiology-Pediatric','Obstetrics','Obsterics&Gynecology-GynecologicOnco']
pediatrics = ['Pediatrics','Pediatrics-CriticalCare','Pediatrics-EmergencyMedicine','Pediatrics-Endocrinology','Pediatrics-Hematology-Oncology', 'Pediatrics-Neurology','Pediatrics-Pulmonology', 'Anesthesiology-Pediatric', 'Cardiology-Pediatric', 'Surgery-Pediatric']
psychic = ['Psychiatry-Addictive', 'Psychology', 'Psychiatry',  'Psychiatry-Child/Adolescent', 'PhysicalMedicineandRehabilitation', 'Osteopath']
neurology = ['Neurology', 'Surgery-Neuro',  'Pediatrics-Neurology', 'Neurophysiology']
surgery = ['Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Maxillofacial', 'Surgery-Plastic', 'Surgery-PlasticwithinHeadandNeck',  'Surgery-Thoracic', 'Surgery-Vascular', 'SurgicalSpecialty', 'Podiatry']
ungrouped = ['Endocrinology','Gastroenterology','Gynecology','Hematology','Hematology/Oncology','Hospitalist','InfectiousDiseases', 'Oncology','Ophthalmology','Otolaryngology','Pulmonology','Radiology']
missing = ['?']

def get_specialty_type(specialty):
    if specialty in pediatrics : return 'pediatrics'
    elif specialty in psychic : return 'psychic'
    elif specialty in neurology : return 'neurology'
    elif specialty in surgery : return 'surgery'
    elif specialty in high_frequency : return 'high_freq'
    elif specialty in low_frequency : return 'low_freq'
    elif specialty in ungrouped : return 'ungrouped'
    elif specialty in missing : return 'missing'

data['medical_specialty'] = data['medical_specialty'].apply(get_specialty_type)

## Diagnoses

In [786]:
def get_diag(diag):
    if '?' in diag: return 'unknown'
    if 'V' in diag:  return 'health_contact'
    if 'E' in diag: return 'injury_poisoning'
    diag_no = int(float(diag))
    if diag_no in range(0, 140): return 'infectious'
    if diag_no in range(140, 240): return 'neoplasms'
    if diag_no == 250: return 'diabetes'
    if diag_no in range(251, 260): return 'endocrine'
    if diag_no in range(240, 280): return 'long_term'
    if diag_no in range(280, 290): return 'blood'
    if diag_no in range(290, 320): return 'mental'
    if diag_no in range(320, 390): return 'nervous'
    if diag_no in range(390, 460): return 'respiratory'
    if diag_no in range(520, 580): return 'digestive'
    if diag_no in range(580, 630): return 'genitourinary'
    if diag_no in range(630, 680): return 'birth'
    if diag_no in range(680, 710): return 'skin'
    if diag_no in range(710, 740): return 'musculoskeletal'
    if diag_no in range(740, 760): return 'congenital'
    if diag_no in range(760, 780): return 'prenital'
    if diag_no in range(780, 800): return 'symptoms'
    if diag_no in range(800, 1000): return 'injury_poisoning'
    return 'other'

In [787]:
data['diagnosis_1'] = data['diag_1'].apply(get_diag)
data['diagnosis_2'] = data['diag_2'].apply(get_diag)
data['diagnosis_3'] = data['diag_3'].apply(get_diag)

In [788]:
def get_diabetes_complications(diag):
    if 'V' in diag or 'E' in diag or '?' in diag: return 'no_diabetes'
    diag_float = round(float(diag),1)
    if diag_float == 250.0: return 'no_complications'
    if diag_float == 250.1: return 'ketoacidosis'
    if diag_float == 250.2: return 'hyperosmolarity'
    if diag_float == 250.3: return 'other_coma'
    if diag_float == 250.4: return 'renal'
    if diag_float == 250.5: return 'ophthalmic'
    if diag_float == 250.6: return 'nuerological'
    if diag_float == 250.7: return 'peripheral'
    if diag_float == 250.8: return 'other'
    if diag_float == 250.9: return 'unspecificed'
    return 'no_diabetes'

In [789]:
data['diabetes_comp_1'] = data['diag_1'].apply(get_diabetes_complications)
data['diabetes_comp_2'] = data['diag_2'].apply(get_diabetes_complications)
data['diabetes_comp_3'] = data['diag_3'].apply(get_diabetes_complications)

In [790]:
def get_diabetes_type(diag):
    if 'V' in diag or 'E' in diag or '?' in diag: return 'no_diabetes'
    if diag[:3] != '250': return 'no_diabetes'
    if diag[-1] in ('0', '2'): return 'type_2'
    else: return 'type_1'

In [791]:
data['diabetes_type_1'] = data['diag_1'].apply(get_diabetes_type)
data['diabetes_type_2'] = data['diag_2'].apply(get_diabetes_type)
data['diabetes_type_3'] = data['diag_3'].apply(get_diabetes_type)

## Readmitted

In [792]:
data['readmitted'] = data['readmitted'].apply(lambda x : 0 if (x == '>30' or x == 'NO') else 1)
data = data.reset_index(drop=True)

## Treatments and Tests

In [793]:
meds_cols = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']

def get_medicine(m):
    if m == 'No' or m == 'Steady':
        return 0
    if m == 'Up' or m == 'Down':
        return 1

def get_diabetes(d):
    if d == 'Yes':
        return 1
    if d == 'No':
        return 0

def get_ch(c):
    if c == 'Ch':
        return 1
    if c == 'No':
        return 0

for m in meds_cols:
    data[m] = data[m].apply(get_medicine)
data['diabetesMed'] = data['diabetesMed'].apply(get_diabetes)
data['change'] = data['change'].apply(get_ch)

In [794]:
def get_pc(p):
    if p == '?':
        return 'unknown'
    if p == 'MD' or p == 'MC':
        return 'medicare'
    if p == 'SP':
        return 'selfpay'
    return 'private_insurance'
data['payer_code'] = data['payer_code'].apply(get_pc)

## Scaling features

In [795]:
def logfunc(m):
    return math.log(m+1)

data['num_medications'] = data['num_medications'].apply(logfunc)
data['num_procedures'] = data['num_procedures'].apply(logfunc)
data['num_lab_procedures'] = data['num_lab_procedures'].apply(logfunc)
data['number_diagnoses'] = data['number_diagnoses'].apply(logfunc)
data['number_emergency'] = data['number_emergency'].apply(logfunc)
data['number_inpatient'] = data['number_inpatient'].apply(logfunc)
data['number_outpatient'] = data['number_outpatient'].apply(logfunc)
data['time_in_hospital'] = data['time_in_hospital'].apply(logfunc)

## Extract new features

In [796]:
data['total_services'] = data['number_inpatient'] + data['number_outpatient'] + data['number_emergency']
data['change_meds'] = np.sum([data[m].astype(int) for m in meds_cols],axis=0)

## Map extra information

In [797]:
admission_type = {
    'Emergency': 1,
    'Urgent': 2,
    'Elective': 3,
    'Newborn': 4,
    'Not available': 5,
    '': 6,
    'Trauma Center': 7,
    'Not Mapped': 8
}
admission_type_reverse = {}

for key in admission_type:
    admission_type_reverse[admission_type[key]] = key
    
data['admission_type_id'] = data['admission_type_id'].replace(admission_type['Urgent'], admission_type['Emergency']) # Urgent -> emergency
data['admission_type_id'] = data['admission_type_id'].replace(admission_type['Trauma Center'],admission_type['Emergency']) # Trauma -> emergency
data['admission_type_id'] = data['admission_type_id'].replace(admission_type[''],admission_type['Not available']) # Not available
data['admission_type_id'] = data['admission_type_id'].replace(admission_type['Not Mapped'],admission_type['Not available']) # Not available
admission_type_concise = {
     1: 'Emergency',
     3: 'Elective',
     4: 'Newborn',
     5: 'Unavailable'
}
data['admission_type'] = data['admission_type_id'].apply(lambda x: admission_type_concise[x])
data = data.drop('admission_type_id', axis=1)

In [798]:
discharge_map = {
    'Discharged home': 1,
    'Discharged to short term hospital': 2,
    'Discharged to skilled nursing facility': 3,
    'Discharged to intermediate care facility': 4,
    'Discharged to inpatient care institution': 5,
    'Discharged to home with home health service': 6,
    'Left against medical advice': 7,
    'Discharged to home under HOME IV provider': 8,
    'Admitted as inpatient to this hospital': 9,
    'Neonate discharged': 10,
    'Expired': 11,
    'Still patient': 12,
    'Hospice at home': 13,
    'Hospice at facility': 14,
    'Transferred to swing bed': 15,
    'Discharged to another institution for outpatient services': 16,
    'Discharged to this institution for outpatient services': 17,
    '': 18,
    'Expired in home': 19,
    'Expired in facility': 20,
    'Expired in unknown place': 21,
    'Discharged to rehab': 22,
    'Transferred to long-term hospital': 23,
    'Discharged to Medicaid but not Medicare-certified facility': 24,
    'Not mapped': 25,
    'Unknown/Invalid': 26,
    'Discharged to federal health care facility': 27,
    'Discharged to psychiatric hospital': 28,
    'Discharged to CAH': 29,
    'Discharged to other health care institution': 30
}


discharge_map_reverse = {}

for key in discharge_map:
    discharge_map_reverse[discharge_map[key]] = key


data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(6,1) # home w/ health services -> home
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(8,1) # 
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(9,1)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(13,1)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(3,2)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(4,2)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(5,2)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(14,2)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(22,2)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(23,2)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(24,2)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(12,10)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(15,10)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(16,10)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(17,10)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(25,18)
data['discharge_disposition_id'] = data['discharge_disposition_id'].replace(26,18)
discharge_disposition_concise = {
     1: 'Discharged home',
     2: 'Discharged hospital',
     7: 'Left AMA',
     10: 'Neonate discharged',
     11: 'Expired',
     18: 'Unavailable',
     19: 'Expired in home',
     20: 'Expired in facility',
     27: 'Discharged federal facility',
     28: 'Discharged psychiatric hospital'
}

data['discharge_disposition'] = data['discharge_disposition_id'].apply(lambda x: discharge_disposition_concise[x])
data = data.drop('discharge_disposition_id', axis=1)


In [799]:
admission_source = {
    'Physician referral': 1,
    'Clinic referral': 2,
    'HMO Referral': 3,
    'Transfer from hospital': 4,
    'Transfer from skilled nursing facility': 5,
    'Transfer from another health facility': 6,
    'Emergency Room': 7,
    'Law enforcement': 8,
    'Not available': 9,
    'Transfer from critical access hospital': 10,
    'Normal delivery': 11,
    'Premature delivery': 12,
    'Sick baby': 13,
    'Extramural birth': 14,
    'Not available': 15,
    '': 17,
    'Transfer from another home health agency': 18,
    'Readmission to same home health agency': 19,
    'Not mapped': 20,
    'Unknown/Invalid': 21,
    'Transfer from hospital inpatient': 22,
    'Born inside this hospital': 23,
    'Born outside this hospital': 24,
    'Transfer from ambulatory surgery center': 25,
    'Transfer from hospice': 26
}

admission_source_reverse = {}

for key in admission_source:
    admission_source_reverse[admission_source[key]] = key


data['admission_source_id'] = data['admission_source_id'].replace(2,1)
data['admission_source_id'] = data['admission_source_id'].replace(3,1)
data['admission_source_id'] = data['admission_source_id'].replace(5,4)
data['admission_source_id'] = data['admission_source_id'].replace(6,4)
data['admission_source_id'] = data['admission_source_id'].replace(10,4)
data['admission_source_id'] = data['admission_source_id'].replace(22,4)
data['admission_source_id'] = data['admission_source_id'].replace(25,4)
data['admission_source_id'] = data['admission_source_id'].replace(15,9)
data['admission_source_id'] = data['admission_source_id'].replace(17,9)
data['admission_source_id'] = data['admission_source_id'].replace(20,9)
data['admission_source_id'] = data['admission_source_id'].replace(21,9)
data['admission_source_id'] = data['admission_source_id'].replace(13,11)
data['admission_source_id'] = data['admission_source_id'].replace(14,11)

admission_source_concise = {
     1: 'Physician referral',
     4: 'Transfer hospital',
     7: 'Emergency Room',
     8: 'Law enforcement',
     9: 'Not available',
     11: 'Delivery'
}
data['admission_source'] = data['admission_source_id'].apply(lambda x: admission_source_concise[x])
data = data.drop('admission_source_id', axis=1)


## One Hot Encoding

In [800]:
from sklearn.preprocessing import MultiLabelBinarizer
def multi_encode(cols):
    to_encode = data[cols].apply(list, axis=1)
    mlb = MultiLabelBinarizer()
    return pd.DataFrame(mlb.fit_transform(to_encode),
                   columns=mlb.classes_,
                   index=to_encode.index)
diagnosis_cols = ['diagnosis_1', 'diagnosis_2', 'diagnosis_3']
comp_cols = ['diabetes_comp_1', 'diabetes_comp_2', 'diabetes_comp_3']
type_cols = ['diabetes_type_1', 'diabetes_type_2', 'diabetes_type_3']
data = data.join(multi_encode(diagnosis_cols).add_prefix('diagnosis_'))
data = data.join(multi_encode(comp_cols).add_prefix('comp_'))
data = data.join(multi_encode(type_cols))

In [801]:
data = data.drop(diagnosis_cols, axis=1)
data = data.drop(comp_cols, axis=1)
data = data.drop(type_cols, axis=1)

In [802]:
cat_vars = ['race', 'gender', 'admission_type', 'discharge_disposition', 'admission_source', 'medical_specialty', 'max_glu_serum', 'A1Cresult', 'payer_code']
for var in cat_vars:
    data = data.join(pd.get_dummies(data[var], prefix=var))
    data = data.drop(var, axis=1)

In [803]:
to_drop = ['encounter_id', 'patient_nbr', 'readmitted', 'diag_1', 'diag_2', 'diag_3', 'weight'] + cat_vars + ['readmitted']
X = data[[c for c in data.columns if c not in to_drop]]
y = data['readmitted']

In [804]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [808]:
output_df = pd.DataFrame(X, copy=True)
output_df['readmitted'] = y
output_df.to_csv('../output/data.csv')

In [697]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [805]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [806]:
clf = RandomForestClassifier(random_state=0).fit(X_train, y_train)
preds = clf.predict(X_test)

In [807]:
accuracy_score(preds, y_test)

0.9377281274195333

In [701]:
preds = clf.predict(X_train)

In [702]:
accuracy_score(preds, y_train)

1.0

In [735]:
pca = PCA(3)

In [736]:
X_pca = X.join(pd.DataFrame(pca.fit_transform(X)))

In [737]:
X_pca

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,metformin,...,A1Cresult_>8,A1Cresult_None,A1Cresult_Norm,payer_code_medicare,payer_code_private_insurance,payer_code_selfpay,payer_code_unknown,0,1,2
0,5,0.693147,3.737670,0.000000,0.693147,0.000000,0.0,0.000000,0.693147,0,...,0,1,0,0,0,0,1,60.967769,-1.292489,1.037986
1,15,1.386294,4.094345,0.000000,2.944439,0.000000,0.0,0.000000,2.302585,0,...,0,1,0,0,0,0,1,50.966457,-0.948403,-1.180779
2,25,1.098612,2.484907,1.791759,2.639057,1.098612,0.0,0.693147,1.945910,0,...,0,1,0,0,0,0,1,40.981763,-0.198489,0.680795
3,35,1.098612,3.806662,0.693147,2.833213,0.000000,0.0,0.000000,2.079442,0,...,0,1,0,0,0,0,1,30.967742,-0.922360,-0.814729
4,45,0.693147,3.951244,0.000000,2.197225,0.000000,0.0,0.000000,1.791759,0,...,0,1,0,0,0,0,1,20.977997,-1.007457,-0.190362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,75,1.386294,3.951244,0.000000,2.833213,0.000000,0.0,0.000000,2.302585,0,...,1,0,0,1,0,0,0,-9.028152,0.208153,-1.172135
101762,85,1.791759,3.526361,1.386294,2.944439,0.000000,0.0,0.693147,2.302585,0,...,0,1,0,1,0,0,0,-19.041908,0.537207,-0.109086
101763,75,0.693147,3.988984,0.000000,2.302585,0.693147,0.0,0.000000,2.639057,0,...,0,1,0,1,0,0,0,-9.027344,1.397790,-0.605530
101764,85,2.397895,3.828641,1.098612,3.091042,0.000000,0.0,0.693147,2.302585,0,...,0,1,0,1,0,0,0,-19.043019,2.445699,-0.913422


In [738]:
X.columns

Index(['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'metformin',
       ...
       'max_glu_serum_None', 'max_glu_serum_Norm', 'A1Cresult_>7',
       'A1Cresult_>8', 'A1Cresult_None', 'A1Cresult_Norm',
       'payer_code_medicare', 'payer_code_private_insurance',
       'payer_code_selfpay', 'payer_code_unknown'],
      dtype='object', length=117)

In [739]:
X.groupby('age').prev_visits.mean()

age
5     0.043478
15    0.476122
25    1.568497
35    0.743576
45    0.756118
55    0.587854
65    0.601121
75    0.577720
85    0.610746
95    0.635875
Name: prev_visits, dtype: float64

In [740]:
X['age'].value_counts()

75    26068
65    22483
55    17256
85    17197
45     9685
35     3775
95     2793
25     1657
15      691
5       161
Name: age, dtype: int64

In [741]:
plt.scatter(X_pca[], X_pca[2], c=X['prev_visits'])

SyntaxError: invalid syntax (<ipython-input-741-40e8c00f3353>, line 1)

In [727]:
print(pca.explained_variance_ratio_)

[0.951562   0.01614143 0.00279377]
